In [5]:
pip install openpyxl


  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/96/c2/3dd434b0108730014f1b96fd286040dc3bcb70066346f7e01ec2ac95865f/et_xmlfile-1.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 1.0 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd

# 제어 문자를 제거하는 함수
def remove_control_characters(s):
    return "".join(ch for ch in s if ch.isprintable())

# 데이터 및 설정
loaded_config = {
    'general': {
        'data_path': '/data/ephemeral/home/data/data'  # 데이터 경로 설정
    }
}
data_path = loaded_config['general']['data_path']

# CSV 파일 불러오기
train_df = pd.read_csv(os.path.join(data_path, 'test.csv'))## 수정

# 'dialogue' 열에 제어 문자 제거 함수 적용
train_df['dialogue'] = train_df['dialogue'].apply(remove_control_characters)
#train_df['summary'] = train_df['summary'].apply(remove_control_characters)
#train_df['topic'] = train_df['topic'].apply(remove_control_characters)

# 확인을 위해 처음 몇 줄 출력
print(train_df.head())

# 필요한 경우 수정된 데이터프레임 저장
train_df.to_csv(os.path.join(data_path, 'test_clean.csv'), index=False)## 수정


    fname                                           dialogue
0  test_0  #Person1#: 더슨 씨, 받아쓰기 좀 해주세요. #Person2#: 네, 실장...
1  test_1  #Person1#: 드디어 왔네! 왜 그렇게 오래 걸렸어?#Person2#: 또 교...
2  test_2  #Person1#: 케이트, 무슨 일이 일어났는지 너는 믿지 못할거야. #Perso...
3  test_3  #Person1#: 생일 축하해, 이건 너를 위한 거야, 브라이언.#Person2#...
4  test_4  #Person1#: 이 올림픽 공원이 정말 크네요!#Person2#: 네. 지금 우...


In [ ]:
import os
import pandas as pd
from openai import OpenAI

# API 설정
client = OpenAI(
    api_key="up_kts6Q2KQWy94negmKwvQcos7cYo6O",  # 실제 API 키 입력
    base_url="https://api.upstage.ai/v1/solar"
)

def translate_text(text, model="solar-1-mini-translate-koen"):
    """ 주어진 텍스트를 영어로 번역합니다. """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": text}
        ],
        stream=False
    )
    return response.choices[0].message.content if response.choices else text

# 데이터 및 설정
loaded_config = {
    'general': {
        'data_path': '/data/ephemeral/home/data/data'  # 데이터 경로 설정
    }
}
data_path = loaded_config['general']['data_path']
train_df = pd.read_csv(os.path.join(data_path, 'train_clean.csv')) ## 수정
batch_size = 100 
columns_to_translate = ['dialogue', 'summary', 'topic']
#columns_to_translate = ['dialogue']

# 배치 3부터 시작하기 위한 계산 (배치 인덱스는 0부터 시작하므로 2 * batch_size)
restart_batch = 8 #6 #64 #83  # fname=train_646이면 restart_batch는 6, 결과물은 batch_7.xslx ## 수정
start_index = restart_batch * batch_size

# 배치 처리 및 Excel 파일 저장
for start in range(start_index, train_df.shape[0], batch_size):
    end = min(start + batch_size, train_df.shape[0])
    batch = train_df.iloc[start:end]
    for column in columns_to_translate:
        batch[column + '_en'] = batch[column].apply(translate_text)
    batch_file = os.path.join(data_path, f'batch_{start // batch_size + 1}.xlsx')
    batch.to_excel(batch_file, index=False)
    print(f"Processed and saved batch {start // batch_size + 1}")

# 모든 배치 파일 병합
all_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.startswith('batch_') and f.endswith('.xlsx')]
all_data = pd.concat((pd.read_excel(f) for f in all_files), ignore_index=True)

# 최종 파일 저장
output_file = os.path.join(data_path, 'test_en.csv') ## 수정
all_data.to_csv(output_file, index=False)
print("All data merged and saved to", output_file)

#18.68시간 예상


In [48]:
batch_9 = pd.read_excel(r'/data/ephemeral/home/data/data/batch_9.xlsx')
batch_9[batch_9['dialogue_en'].isna()==True]

,fname,dialogue,summary,topic,dialogue_en,summary_en,topic_en


In [45]:
batch_7 = pd.read_excel(r'/data/ephemeral/home/data/data/batch_7.xlsx')
batch_7[batch_7['dialogue_en'].isna()==True]

,fname,dialogue,summary,topic,dialogue_en,summary_en,topic_en


In [17]:
batch_84 = pd.read_excel(r'/data/ephemeral/home/data/data/batch_84.xlsx')
batch_84[batch_84['dialogue_en'].isna()==True]

,fname,dialogue,summary,topic,dialogue_en,summary_en,topic_en


In [22]:
batch_65 = pd.read_excel(r'/data/ephemeral/home/data/data/batch_65.xlsx')
batch_65[batch_65['dialogue_en'].isna()==True]

,fname,dialogue,summary,topic,dialogue_en,summary_en,topic_en


In [29]:
print("대체할 train_en의 행 수:", sum(mask))
print("batch_65의 행 수:", len(batch_65))


대체할 train_en의 행 수: 109
batch_65의 행 수: 100


In [32]:
import pandas as pd

# 데이터 로드 (예제)
train_en = pd.read_csv(r'/data/ephemeral/home/data/data/train_en.csv')

# 삭제할 행 식별
mask = train_en['fname'].apply(lambda x: 'train_6400' <= x <= 'train_6499')

# 삭제할 위치의 최소 인덱스 계산
if mask.any():  # mask에 True가 하나라도 있는 경우
    start_index = train_en.index[mask].min()

    # 해당 행들 삭제
    train_en = train_en[~mask]

    # batch_65 데이터 인덱스 조정
    batch_65.index = range(start_index, start_index + len(batch_65))

    # train_en과 batch_65 결합
    train_en = pd.concat([train_en, batch_65]).sort_index()
else:
    print("해당 범위의 fname을 가진 행이 없습니다.")

# 변경된 데이터프레임 저장
train_en.to_csv('train_en_65.csv', index=False)



In [35]:
import pandas as pd

# 데이터 로드 (예제)
train_en_65 = pd.read_csv(r'/data/ephemeral/home/notebook/train_en_65.csv')

# 삭제할 행 식별
mask = train_en_65['fname'].apply(lambda x: 'train_8300' <= x <= 'train_8399')

# 삭제할 위치의 최소 인덱스 계산
if mask.any():  # mask에 True가 하나라도 있는 경우
    start_index = train_en_65.index[mask].min()

    # 해당 행들 삭제
    train_en_65 = train_en_65[~mask]

    # batch_65 데이터 인덱스 조정
    batch_84.index = range(start_index, start_index + len(batch_84))

    # train_en_65과 batch_65 결합
    train_en_84 = pd.concat([train_en_65, batch_84]).sort_index()
else:
    print("해당 범위의 fname을 가진 행이 없습니다.")

# 변경된 데이터프레임 저장
train_en_84.to_csv('train_en_84.csv', index=False)



In [36]:
train_en_84[train_en_84['dialogue_en'].isna()==True]

,fname,dialogue,summary,topic,dialogue_en,summary_en,topic_en


In [37]:
# 변경된 데이터프레임 저장
train_en_84.to_csv('train_en.csv', index=False)